# Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import random
global_lambdas = []
global_gammas = []

# Generate data

In [ ]:
mu, sigma = 0, 1 # mean and standard deviation
N = 1000

### Generate z_vec

Generate vector z of hidden states as follows:

$$z_{i} = \gamma z_{i-1} + \delta_i \hspace{0.5cm} \text{with } z_0 = 0, \hspace{0.2cm} \gamma = 0.5, \hspace{0.2cm} \delta = norm(0, 1)$$

In [ ]:
z0 = 0
gamma = 0.5
delta = np.random.normal(mu, sigma, N)
z_vec = []
z_vec.append(gamma*z0 + delta[0])
for i in range(1, N):
    z_vec.append(gamma*z_vec[i-1] + delta[i])  

# convert to np array
z_vec = np.array(z_vec)

### Generate x_vec

Generate vector x of observations as follows:

$$x_{i} = norm(z_{i},1) \hspace{0.5cm}$$

In [ ]:
x_vec = []
for i in range(N):
    x_i = np.random.normal(z_vec[i], sigma)
    #x_i = z_vec[i]
    x_vec.append(x_i)

# convert to np array
x_vec = np.array(x_vec)

# Algorithm

$$ \underset{z}{min} \sum_{i} \left[\left(x_i - z_i \right)^2 + \lambda \left(z_i - \gamma z_{i-1}\right)^2\right] = \sum_{i=1}^{m} \left(x_i - z_i \right)^2 + \sum_{i=0}^{m-1}\lambda \left(y_{i+1} - \gamma z_{i}\right)^2$$

Taking derivative over $z_i$ gives us:
$$ -2\left(x_i - z_i \right) - 2\lambda \left(y_{i+1} - \gamma z_{i}\right) = 0 \Leftrightarrow z_i = \frac{2\lambda y_{i+1} + 2x_i}{2 + 2\gamma}$$


**Step 1**: Estimate $\gamma$

**Step 2**: Estimate $z$ from the just-found $\gamma$

In [ ]:
def step_one(lmda, gamma, x_vec, N, prev_z=None):
    for i in range(N):
        

In [ ]:
def step_two(z):
    Z = []
    Z.append(z0)
    for i in range(N-1):
        Z.append(z[i])
    Z = np.matrix(Z, dtype='float')
    ZZ = np.matmul(Z, Z.transpose())
    ZZ = np.linalg.pinv(ZZ)
    ZZ_vec = np.matmul(Z, z.transpose())
    gamma_vec = ZZ*ZZ_vec
    gamma = gamma_vec.item(0)
    return gamma